In [1]:
import multiprocessing

In [2]:
from multiprocessing.managers import BaseManager

In [3]:
from htm.bindings.algorithms import Connections

In [4]:
from htm.bindings.sdr import SDR

In [5]:
import random

In [49]:
def pollute(c, segments_per_cell, synapses_per_segment):
    for cell in range(c.numCells()):
        for _ in range(segments_per_cell):
            seg = c.createSegment(cell, segments_per_cell)
            p_cells = random.sample(range(c.numCells()), k=synapses_per_segment)
            for p_cell in p_cells:
                c.createSynapse(seg, p_cell, 0.0)

In [50]:
def info(c):
    for seg in range(c.numSegments()):
        print(f'segment {seg}:')
        synapses = c.synapsesForSegment(seg)
        for syn in synapses:
            print(f'\t syn {syn}: p: {c.permanenceForSynapse(syn)} p_cell: {c.presynapticCellForSynapse(syn)}')

In [51]:
def learn(c, segments, active_cells):
    for seg in segments: 
        c.adaptSegment(seg, active_cells, 0.01, 0.01, False)

In [52]:
class MyManager(BaseManager):
    pass

MyManager.register('Connections', Connections)

In [53]:
def learn_parallel(c, segments, active_cells, n_cores=2):
    processes = list()
    batch_size = len(segments)//n_cores
    for i in range(n_cores):
        p = multiprocessing.Process(target=learn, args=(c, segments[batch_size*i: batch_size*(i+1)], active_cells))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()

In [66]:
c = Connections(5000, 1)
pollute(c, 2, 2)
# info(c)

In [67]:
segments = list(range(c.segmentFlatListLength()))
active_cells = SDR(c.numCells())
active_cells.sparse = c.presynapticCellsForSegment(random.randint(0, c.segmentFlatListLength()))

In [68]:
%%time
learn(c, segments, active_cells)

CPU times: user 10.5 ms, sys: 44 µs, total: 10.5 ms
Wall time: 10.4 ms


In [73]:
manager = MyManager()

In [74]:
manager.start()

In [75]:
c = manager.Connections(5000, 1)
pollute(c, 2, 2)
# info(c)

In [76]:
segments = list(range(c.segmentFlatListLength()))
active_cells = SDR(c.numCells())
active_cells.sparse = c.presynapticCellsForSegment(random.randint(0, c.segmentFlatListLength()))

In [77]:
%%time
learn_parallel(c, segments, active_cells, n_cores=4)

CPU times: user 3.53 ms, sys: 47.5 ms, total: 51.1 ms
Wall time: 891 ms


In [78]:
manager.shutdown()